In [ ]:
Oracle
SAP

In [1]:
import requests
from bs4 import BeautifulSoup
import time
from tqdm import tqdm
from pprint import pprint
import random
import json
from scraping_utils import scrape_resume, scrape_resume_links
from concurrent.futures import ThreadPoolExecutor, as_completed

# lxml parser
1. create an HTTP session using requests.Session()
2. send a GET request to the given url
3. check for request errors using raise_for_status()
4. parse the HTML response with BeautifulSoup using the "lxml" parser
5. find all <div> elements with class "media-body" or "single-post-body"
6. Return the list of matching <div> elements

In [2]:
def lxml_parser(url: str) -> list:
    session = requests.Session()

    response = session.get(url)
    response.raise_for_status()

    soup = BeautifulSoup(response.text, "lxml")
    data = {}

    target_divs = soup.find_all("div", class_=["media-body", "single-post-body"])

    return target_divs    

In [3]:
# helper
def loop_prnt(url):
    for div in lxml_parser(url):
        print(div)

### Scraped LXML format:
- summary: list of string
- skills: dict. keys and values are strings
- experience: list of dict. resposibilities: dict

### Expected output of URL:
https://www.hireitpeople.com/resume-database/63-net-developers-architects-resumes/628290-sr-crm-developer-resume-saint-louis-mo

```json
{
    "SUMMARY": [
        "Experienced MSCRM Techno Functional Consultant with 7 years of experience...",
        "Experienced Sr.Developer in MSCRM, ASP.NET.",
        .
        .
        .
    ],
    "SKILLS": {
        "CRM": "Microsoft DynamicsCRM 2016/2015/2013...",
        "Languages": "C, C++, C#, VB.NET, ADO.NET, SQL",
        ...
        ...
        ...
    }
    "PROFESSIONAL EXPEREINCE":[ 
        {
            "company_name": "Confidential, Saint Louis, MO",
            "job_role": "Sr.CRM Developer",
            "responsibilities": [
                "Worked on project-based directives for client... ",
                "Assist in Upgrade/Migration from CRM & 2015... ",
                .
                .
                .
            ],
            "environment": "MSCRM 2015, On-Premises Deployment C#, Plugin Registrations... "
        },
        {
            "company_name": "Confidential, St. Louis, MO",
            "job_role": "SR.CRM Developer",
            "responsibilities": [
                "Worked with n-Tier Architecture involving Services Layer...",
                "Worked on project-based directives for client CRM environments.",
                .
                .
                .
            ],
            "environment": "MSCRM 2013, On-Premises Deployment, C#..."
        },
        {
        .
        },
        {
        .
        },
    ]
}

In [1]:
url = "https://www.hireitpeople.com/resume-database/63-net-developers-architects-resumes/628290-sr-crm-developer-resume-saint-louis-mo"
loop_prnt(url)
print(scrape_resume(url))

NameError: name 'loop_prnt' is not defined

In [5]:
"""
-summary: single string
-skills: current technological skills. absence of <strong> and <u> tags. list of strings
-experience: professional career. absence of strong and u tags. missing job_role, company_name

expected output ->
{
    "SUMMARY": [
        "Information Technology System professional with...",
        "Strong analytical and problem - solving skills.",
        .
        .
        .
    ],
    "SKILLS": {
        "Languages": "C#, Java(Android), Flex 3... ",
        "Databases": "SQL Server 2008(2000/2005), MySQL, SQLite",
        ...
        ...
        ...
    }
    "PROFESSIONAL EXPEREINCE":[ 
        {
            "company_name": "",
            "job_role": "Senior Software Developer/Lead Architect",
            "responsibilities": [
                "Responsible for IT operations servising law firms... ",
                "Leadership (player/coach) of a 10 person development team.",
                .
                .
                .
            ],
            "environment": ""
        },
    ]
}
"""
url = "https://www.hireitpeople.com/resume-database/63-net-developers-architects-resumes/120-net-architect-senior-developer"
loop_prnt(url)

<div class="media-body">
<div>
<h3>.net Architect/senior Developer Resume</h3>
<div class="resume-rated mt-0">
<div class="rating-group">
<input class="rating__input rating__input--none" disabled="" id="rated-none" readonly="" type="radio" value="0"/>
<label aria-label="1 star" class="rating__label" for="rated-1"><i class="rating__icon rating__icon--star fa fa-star"></i></label>
<input class="rating__input" disabled="" id="rated-1" readonly="" type="radio" value="1"/>
<label aria-label="2 stars" class="rating__label" for="rated-2"><i class="rating__icon rating__icon--star fa fa-star"></i></label>
<input checked="" class="rating__input" disabled="" id="rated-2" readonly="" type="radio" value="2"/>
<label aria-label="3 stars" class="rating__label" for="rated-3"><i class="rating__icon rating__icon--star fa fa-star"></i></label>
<input class="rating__input" disabled="" id="rated-3" readonly="" type="radio" value="3"/>
<label aria-label="4 stars" class="rating__label" for="rated-4"><i class

In [6]:
"""
-summary: single big string, but is a point
-skills: core competencies
-education: academic profile. task irrelavant
-project: task irrelavant
-responsibilities: single string or list of strings. missing sub-heading

expected output ->
{
    "SUMMARY": [
        "Software Professional with more than 10+ years...",
        "In 10 years experiences in Java and J2EE...",
        .
        .
        .
    ],
    "SKILLS": {
        "Operating Systems": "MS-DOS 6.0, Sco-UNIX, Windows...",
        "Languages": "PI/SQL, COBOL, Pascal, Java",
        ...
        ...
        ...
    }
    "PROFESSIONAL EXPEREINCE":[ 
        {
            "company_name": "Confidential",
            "job_role": "Senior Developer",
            "responsibilities": [
                "Created theme and skins for ISA application... ",
                "Involved in front end development... ",
                .
                .
                .
            ],
            "environment": "Java, IBM websphere portal server 6.0, Struts... "
        },
        {
            "company_name": "Confidential",
            "job_role": "Senior Developer",
            "responsibilities": [
                "The system has maintained the insured lossed that have... ",
                "These types of claims have a tremendous effect on a... ",
                .
                .
                .
            ],
            "environment": "Java, Spring, XML, XSD, Subversion... "
        },
        {
        .
        },
        {
        .
        },
    ]
}
"""

url = "https://www.hireitpeople.com/resume-database/64-java-developers-architects-resumes/1540-java-devolper-resume"
loop_prnt(url)

<div class="media-body">
<div>
<h3>Java Devolper Resume</h3>
<div class="resume-rated mt-0">
<div class="rating-group">
<input class="rating__input rating__input--none" disabled="" id="rated-none" readonly="" type="radio" value="0"/>
<label aria-label="1 star" class="rating__label" for="rated-1"><i class="rating__icon rating__icon--star fa fa-star"></i></label>
<input class="rating__input" disabled="" id="rated-1" readonly="" type="radio" value="1"/>
<label aria-label="2 stars" class="rating__label" for="rated-2"><i class="rating__icon rating__icon--star fa fa-star"></i></label>
<input checked="" class="rating__input" disabled="" id="rated-2" readonly="" type="radio" value="2"/>
<label aria-label="3 stars" class="rating__label" for="rated-3"><i class="rating__icon rating__icon--star fa fa-star"></i></label>
<input class="rating__input" disabled="" id="rated-3" readonly="" type="radio" value="3"/>
<label aria-label="4 stars" class="rating__label" for="rated-4"><i class="rating__icon rat